In [1]:
## imporrt libarary 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [2]:
# Define the data as a dictionary
new_data_test = pd.DataFrame({
    "HallwayType": ["terraced"],
    "TimeToSubway": ["0-5min"],
    "SubwayStation": ["Kyungbuk_uni_hospital"],
    "N_FacilitiesNearBy(ETC)": [0.0],
    "N_FacilitiesNearBy(PublicOffice)": [5.0],
    "N_SchoolNearBy(University)": [3.0],
    "N_Parkinglot(Basement)": [930.0],
    "YearBuilt": [2013],
    "N_FacilitiesInApt": [7],
    "Size(sqf)": [910]
})
new_data_test

,HallwayType,TimeToSubway,SubwayStation,N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(PublicOffice),N_SchoolNearBy(University),N_Parkinglot(Basement),YearBuilt,N_FacilitiesInApt,Size(sqf)
0,terraced,0-5min,Kyungbuk_uni_hospital,0.0,5.0,3.0,930.0,2013,7,910


In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/edwardsembiring1000/edo-020/sa-development.json"

In [4]:
project_id = 'dti-ds'
dataset_id = 'edo_dataset_020'
table_id = 'x_test_cleaned'
region = 'us-central1'
bucket_name = 'edo_gcs_020'
blob_name = 'data/x_test_cleaned.csv'

model_name = 'best_model.pkl'

In [5]:
# Upload the model to Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob('model/best_model.pkl')
    blob_model.upload_from_filename('best_model.pkl')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [6]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

#query
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
daegu_load_test = query_job.result().to_dataframe()

daegu_load_test.columns = new_data_test.columns

daegu_load_test

/home/edwardsembiring1000/miniconda3/envs/daegu-env/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,HallwayType,TimeToSubway,SubwayStation,N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(PublicOffice),N_SchoolNearBy(University),N_Parkinglot(Basement),YearBuilt,N_FacilitiesInApt,Size(sqf)
0,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,779
1,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903
2,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903
3,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903
4,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903
...,...,...,...,...,...,...,...,...,...,...
820,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,644
821,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,914
822,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,914
823,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,914


In [8]:
import pickle

with open ('best_model.pkl','rb') as f:
    loaded_model = pickle.load(f)

y_pred_cloud = loaded_model.predict(new_data_test)
y_pred_cloud

array([293533.38], dtype=float32)

In [9]:
new_data_test_2 = new_data_test.copy()
new_data_test_2['Predicition'] = y_pred_cloud
new_data_test_2

,HallwayType,TimeToSubway,SubwayStation,N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(PublicOffice),N_SchoolNearBy(University),N_Parkinglot(Basement),YearBuilt,N_FacilitiesInApt,Size(sqf),Predicition
0,terraced,0-5min,Kyungbuk_uni_hospital,0.0,5.0,3.0,930.0,2013,7,910,293533.375


In [10]:
y_pred = loaded_model.predict(daegu_load_test)
y_pred[:10]

array([151206.69, 162127.62, 162127.62, 162127.62, 162127.62, 162127.62,
       162127.62, 162127.62,  92636.59,  98346.77], dtype=float32)

In [11]:
daegu_load_test['Prediction'] = y_pred
daegu_load_test

,HallwayType,TimeToSubway,SubwayStation,N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(PublicOffice),N_SchoolNearBy(University),N_Parkinglot(Basement),YearBuilt,N_FacilitiesInApt,Size(sqf),Prediction
0,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,779,151206.687500
1,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903,162127.625000
2,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903,162127.625000
3,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903,162127.625000
4,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903,162127.625000
...,...,...,...,...,...,...,...,...,...,...,...
820,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,644,119690.242188
821,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,914,151006.140625
822,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,914,151006.140625
823,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,914,151006.140625


In [12]:
from google.cloud import bigquery

# Define a function to clean column names
def clean_column_name(column_name):
    return ''.join([char if char.isalnum() or char == '_' else '_' for char in column_name])

table_id = 'unseen_daegu_predicted'

# Construct a bigquery client object
client = bigquery.Client()

# define the full table id
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

# Apply the function to clean all column names
daegu_load_test.columns = [clean_column_name(col) for col in daegu_load_test.columns]

job = client.load_table_from_dataframe(daegu_load_test,table_full_id)

job.result()
print(f"Loaded {job.output_rows} rows into {table_full_id}")

Loaded 825 rows into dti-ds.edo_dataset_020.unseen_daegu_predicted
